# Edgar Sabán - 19012631

# Preparación de los datos

In [11]:
#Carga de librerías
import numpy as np
import tensorflow.compat.v1 as tf
import pandas as pd
import matplotlib.pyplot as plt
import datetime, os
%load_ext tensorboard
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
# Carga de datos y conversión a un DataFrame
data = np.load('data/proyecto_training_data.npy')
data = pd.DataFrame(data)
data.columns = ["SalePrice", "OverallQual", "1stFlrSF", "TotRmsAbvGrd", "YearBuilt", "LotFrontage"]
data.describe()

,SalePrice,OverallQual,1stFlrSF,TotRmsAbvGrd,YearBuilt,LotFrontage
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1201.000000
mean,180921.195890,6.099315,1162.626712,6.517808,1971.267808,70.049958
std,79442.502883,1.382997,386.587738,1.625393,30.202904,24.284752
min,34900.000000,1.000000,334.000000,2.000000,1872.000000,21.000000
25%,129975.000000,5.000000,882.000000,5.000000,1954.000000,59.000000
50%,163000.000000,6.000000,1087.000000,6.000000,1973.000000,69.000000
75%,214000.000000,7.000000,1391.250000,7.000000,2000.000000,80.000000
max,755000.000000,10.000000,4692.000000,14.000000,2010.000000,313.000000


In [13]:
# Dividiendo datos para entrenamiento y test
data = data.sample(frac = 1)
index = int(0.8 * len(data))
df_train = data[:index]
df_test = data[:-index]

# Modelo de regresión lineal con Tensor Flow

En el proyecto anterior utilicé la variable **OverallQual**, debido a que tenía una mejor correlación respecto a la varible **SalePrice**.

Con base a la teoría de Gradient Descent se implementa el algoritmo con apoyo de **Tensor Flow**.

In [14]:
# MRL = modelo de regresión lineal
class MRL:
    
    # Constructor de clase para la inicialización de los parámetros
    def __init__ (self):
        self.parametros = tf.get_variable("parametros", dtype = tf.float32, shape = [1,2], initializer = tf.zeros_initializer())
    
    # Cálculo de MSE
    def MSE(self, x, y):
        error = 1/2 * tf.reduce_mean(tf.math.square(y - tf.matmul(self.parametros,x)))
        return error

    # Cálculo de  gradient descent
    def gradient_descent(self, x, y, lr):
        mse = self.MSE(x,y)
        error_summary = tf.summary.scalar("mse_summary", mse)
        gradiente = tf.gradients(mse, [self.parametros])
        new_w = tf.assign(self.parametros, self.parametros -lr * gradiente[0])
        return new_w, mse, error_summary

In [15]:
def training_model(x, y, lr, epochs, printl):
    # String para experimento
    string = './logs/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S") +"_epochs="+str(epochs)+"_lr="+str(lr)

    # Definición del grafo
    graph = tf.Graph()
    with graph.as_default():
        
        # Creación de los tensores input
        tx = tf.placeholder(tf.float32, [2,len(df_train["OverallQual"])], "tx")
        ty = tf.placeholder(tf.float32, [len(df_train["SalePrice"])], "ty")
        
        # Creación del nodo para el cálculo de los nuevos parámetros según gradient_descent
        mrl = MRL()
        nuevos_parametros = mrl.gradient_descent(tx, ty, lr)
        
        # Writer para utilizacion de tensorboard
        writer = tf.summary.FileWriter(string, graph)
        
        # Ejecución del grafo
        with tf.train.MonitoredSession() as session:
            # Paso de valores a los tensores input
            tensor_dict = {tx:x, ty:y}
            
            # Ejecución de epochs + 1 veces del algoritmo
            for i in range(epochs + 1):
                # Entrenamiento del modelo para cálculo de los nuevos parámetros
                entrenamiento = session.run(nuevos_parametros, feed_dict = tensor_dict)
                # Imprimiendo los epoch según el printl
                if i % printl == 0:
                    parametros = session.run(mrl.parametros, feed_dict = tensor_dict)
                    # Agregar datos a writer para poder visualizarlos en tensorboard
                    writer.add_summary(entrenamiento[2], i)
                    print("Epoch: ", i, "Parámetros: ", parametros, "mse: ", entrenamiento[1])
                    
            writer.close()

## Cálculo de experimentos

### Hipótesis

Según el algoritmo de Gradient Descent para alcanzar el error mínino (MSE) se depende del Learning rate, lo cual define la cantidad de epochs que son necesarios para alcanzar la convergencia.

Se estima que el lr debe estar comprendido entre 0.01, 0.02, 0.03, 0.04, 0.05, 0.06

In [16]:
# Preparando X y Y para entrenar modelo
x = df_train["OverallQual"]
# Se agrega un 1 a X para que pueda realizarse los cálculos del gradiente descent
x = np.array([x, np.ones_like(x)], dtype = "float64")
y = df_train["SalePrice"]

In [22]:
#Experimento 1
training_model(x, y, 0.01, 24000, 1500)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Parámetros:  [[11909.917   1811.4465]] mse:  19658408000.0
Epoch:  1500 Parámetros:  [[ 38034.73  -47911.402]] mse:  1284399000.0
Epoch:  3000 Parámetros:  [[ 41989.395 -73206.41 ]] mse:  1238740100.0
Epoch:  4500 Parámetros:  [[ 43893.934 -85388.3  ]] mse:  1228150500.0
Epoch:  6000 Parámetros:  [[ 44811.145 -91255.016]] mse:  1225694200.0
Epoch:  7500 Parámetros:  [[ 45252.87 -94080.42]] mse:  1225124400.0
Epoch:  9000 Parámetros:  [[ 45465.605 -95441.1  ]] mse:  1224992400.0
Epoch:  10500 Parámetros:  [[ 45568.06 -96096.39]] mse:  1224961800.0
Epoch:  12000 Parámetros:  [[ 45617.4  -96411.99]] mse:  1224954600.0
Epoch:  13500 Parámetros:  [[ 45641.15 -96563.91]] mse:  1224953000.0
Epoch:  15000 Parámetros:  [[ 45652.58 -96637.04]] mse:  1224952600.0
Epoch:  16500 Parámetros:  [[ 45658.04 -96671.95]] mse:  1224952400.0
Epoch:  18000 Parámetros:  [[ 45660.6

In [24]:
#Experimento 2
training_model(x, y, 0.02, 22000,1500)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Parámetros:  [[23819.834  3622.893]] mse:  19658408000.0
Epoch:  1500 Parámetros:  [[ 41992.48 -73226.13]] mse:  1238730200.0
Epoch:  3000 Parámetros:  [[ 44812.18 -91261.63]] mse:  1225693200.0
Epoch:  4500 Parámetros:  [[ 45465.93  -95443.164]] mse:  1224992100.0
Epoch:  6000 Parámetros:  [[ 45617.5  -96412.67]] mse:  1224954600.0
Epoch:  7500 Parámetros:  [[ 45652.65 -96637.47]] mse:  1224952400.0
Epoch:  9000 Parámetros:  [[ 45660.754 -96689.31 ]] mse:  1224952600.0
Epoch:  10500 Parámetros:  [[ 45662.605 -96701.17 ]] mse:  1224952600.0
Epoch:  12000 Parámetros:  [[ 45662.605 -96701.17 ]] mse:  1224952600.0
Epoch:  13500 Parámetros:  [[ 45662.605 -96701.17 ]] mse:  1224952600.0
Epoch:  15000 Parámetros:  [[ 45662.605 -96701.17 ]] mse:  1224952600.0
Epoch:  16500 Parámetros:  [[ 45662.605 -96701.17 ]] mse:  1224952600.0
Epoch:  18000 Parámetros:  [[ 45662

In [26]:
#Experimento 3
training_model(x, y, 0.03, 10000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Parámetros:  [[35729.75    5434.3394]] mse:  19658408000.0
Epoch:  1000 Parámetros:  [[ 41995.586 -73246.016]] mse:  1238720600.0
Epoch:  2000 Parámetros:  [[ 44813.207 -91268.195]] mse:  1225691800.0
Epoch:  3000 Parámetros:  [[ 45466.24  -95445.164]] mse:  1224992400.0
Epoch:  4000 Parámetros:  [[ 45617.594 -96413.24 ]] mse:  1224954500.0
Epoch:  5000 Parámetros:  [[ 45652.676 -96637.65 ]] mse:  1224952700.0
Epoch:  6000 Parámetros:  [[ 45660.785 -96689.516]] mse:  1224952600.0
Epoch:  7000 Parámetros:  [[ 45662.688 -96701.69 ]] mse:  1224952400.0
Epoch:  8000 Parámetros:  [[ 45662.83 -96702.59]] mse:  1224952600.0
Epoch:  9000 Parámetros:  [[ 45662.83 -96702.59]] mse:  1224952600.0
Epoch:  10000 Parámetros:  [[ 45662.83 -96702.59]] mse:  1224952600.0


In [27]:
#Experimento 4
training_model(x, y, 0.04, 10000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Parámetros:  [[47639.668  7245.786]] mse:  19658408000.0
Epoch:  1000 Parámetros:  [[ 43412.527 -82309.09 ]] mse:  1230142300.0
Epoch:  2000 Parámetros:  [[ 45342.984 -94656.78 ]] mse:  1225057500.0
Epoch:  3000 Parámetros:  [[ 45617.684 -96413.83 ]] mse:  1224954800.0
Epoch:  4000 Parámetros:  [[ 45656.773 -96663.87 ]] mse:  1224952400.0
Epoch:  5000 Parámetros:  [[ 45662.35 -96699.52]] mse:  1224952400.0
Epoch:  6000 Parámetros:  [[ 45662.94 -96703.31]] mse:  1224952700.0
Epoch:  7000 Parámetros:  [[ 45662.94 -96703.31]] mse:  1224952700.0
Epoch:  8000 Parámetros:  [[ 45662.94 -96703.31]] mse:  1224952700.0
Epoch:  9000 Parámetros:  [[ 45662.94 -96703.31]] mse:  1224952700.0
Epoch:  10000 Parámetros:  [[ 45662.94 -96703.31]] mse:  1224952700.0


In [28]:
#Experimento 5
training_model(x, y, 0.05, 10000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Parámetros:  [[59549.59   9057.233]] mse:  19658408000.0
Epoch:  1000 Parámetros:  [[ 44811.812 -87790.17 ]] mse:  1232685400.0
Epoch:  2000 Parámetros:  [[ 45552.086 -95932.45 ]] mse:  1224969200.0
Epoch:  3000 Parámetros:  [[ 45652.92 -96637.98]] mse:  1224952700.0
Epoch:  4000 Parámetros:  [[ 45662.324 -96699.58 ]] mse:  1224952600.0
Epoch:  5000 Parámetros:  [[ 45663.062 -96704.42 ]] mse:  1224952400.0
Epoch:  6000 Parámetros:  [[ 45663.062 -96704.42 ]] mse:  1224952400.0
Epoch:  7000 Parámetros:  [[ 45663.062 -96704.42 ]] mse:  1224952400.0
Epoch:  8000 Parámetros:  [[ 45663.062 -96704.42 ]] mse:  1224952400.0
Epoch:  9000 Parámetros:  [[ 45663.062 -96704.42 ]] mse:  1224952400.0
Epoch:  10000 Parámetros:  [[ 45663.062 -96704.42 ]] mse:  1224952400.0


In [30]:
#Experimento 6
training_model(x, y, 0.06, 10000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Parámetros:  [[71459.5   10868.679]] mse:  19658408000.0
Epoch:  1000 Parámetros:  [[nan nan]] mse:  nan
Epoch:  2000 Parámetros:  [[nan nan]] mse:  nan
Epoch:  3000 Parámetros:  [[nan nan]] mse:  nan
Epoch:  4000 Parámetros:  [[nan nan]] mse:  nan
Epoch:  5000 Parámetros:  [[nan nan]] mse:  nan
Epoch:  6000 Parámetros:  [[nan nan]] mse:  nan
Epoch:  7000 Parámetros:  [[nan nan]] mse:  nan
Epoch:  8000 Parámetros:  [[nan nan]] mse:  nan
Epoch:  9000 Parámetros:  [[nan nan]] mse:  nan
Epoch:  10000 Parámetros:  [[nan nan]] mse:  nan


In [19]:
%tensorboard --logdir ./logs

Reusing TensorBoard on port 6006 (pid 9373), started 0:01:29 ago. (Use '!kill 9373' to kill it.)

## Conclusiones

1. Al observar el comportamiento del MSE respecto al Learning Rate, se puede observar que el error mínimo se alcanza cuanto más grande es el Learning Rate.

2. Mientras más grande es el Learning Rate, los pasos del gradient descent son más grandes por lo cual se deben utilizar menos Epochs.

3. Mientas más grande es el Learning Rate, se observar que el algortimo gradient descent da como resultados nan.

4. Para converger de manera más rápida y con menos epochs, es necesario definir el hyperparámetros lr = 0.04, lo cual obtiene el error minímo aproximadamente en el epoch = 4000